<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABCD_ASPNETCORE/blob/main/sql_%D0%B7%D0%B0%D0%B4%D0%B0%D1%87%D0%B8_%D0%BD%D0%B0_%D1%81%D0%BE%D0%B1%D0%B5%D1%81%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Задача 1: Рассчитать среднее время сессии на веб-страницах**

Даны две таблицы: `Customers` (Клиенты) и `PageEvents` (События на странице). В таблицах содержится следующая информация:

---

#### Таблица `Customers`  
Содержит данные о клиентах.

| **Название поля** | **Тип данных**    | **Описание**                          |
|--------------------|-------------------|---------------------------------------|
| `Id`              | `INT`            | Уникальный идентификатор клиента.     |
| `Name`            | `NVARCHAR(255)`  | Имя клиента.                          |
| `Email`           | `NVARCHAR(255)`  | Электронная почта клиента.            |

---

#### Таблица `PageEvents`  
Содержит записи о действиях клиентов на страницах сайта.

| **Название поля** | **Тип данных**    | **Описание**                                                                 |
|--------------------|-------------------|------------------------------------------------------------------------------|
| `Id`              | `INT`            | Уникальный идентификатор события.                                           |
| `PageName`        | `NVARCHAR(255)`  | Название страницы, на которой произошло событие.                            |
| `CustomerId`      | `INT`            | Идентификатор клиента (ссылка на поле `Id` из таблицы `Customers`).         |
| `Timestamp`       | `DATETIME`       | Время, когда произошло событие.                                             |
| `Type`            | `NVARCHAR(10)`   | Тип события: `entry` (вход на страницу) или `exit` (выход со страницы).     |

---

#### Задание

1. Для каждой страницы рассчитать среднее время сессии.  
2. Сессия определяется как разница во времени между событием входа (`entry`) и выхода (`exit`) для одной страницы и одного клиента.  
3. Если на одной странице для одного клиента записано несколько входов и выходов, считать сессиями события, упорядоченные по времени.  
4. Результат должен включать название страницы и среднее время сессии (в минутах).  

---

#### Пример данных

##### Таблица `Customers`:

| **Id** | **Name**    | **Email**           |
|--------|-------------|---------------------|
| 1      | John Doe    | john@example.com    |
| 2      | Jane Smith  | jane@example.com    |

##### Таблица `PageEvents`:

| **Id** | **PageName**    | **CustomerId** | **Timestamp**         | **Type** |
|--------|------------------|----------------|-----------------------|----------|
| 1      | HomePage         | 1              | 2024-12-26 09:00:00  | entry    |
| 2      | HomePage         | 1              | 2024-12-26 09:15:00  | exit     |
| 3      | HomePage         | 2              | 2024-12-26 10:00:00  | entry    |
| 4      | HomePage         | 2              | 2024-12-26 10:30:00  | exit     |
| 5      | ContactPage      | 1              | 2024-12-26 11:00:00  | entry    |
| 6      | ContactPage      | 1              | 2024-12-26 11:20:00  | exit     |

---

#### Ожидаемый результат

| **PageName**     | **AvgSessionTime (min)** |
|-------------------|--------------------------|
| HomePage          | 22.5                    |
| ContactPage       | 20.0                    |

#Ответ


```sql
CREATE TABLE Customers (
    Id INT PRIMARY KEY NOT NULL,
    Name NVARCHAR(255) NOT NULL,
    Email NVARCHAR(255) NOT NULL
);

CREATE TABLE PageEvents (
    Id INT PRIMARY KEY NOT NULL,
    PageName NVARCHAR(255) NOT NULL,
    CustomerId INT NOT NULL,
    Timestamp DATETIME NOT NULL,
    Type NVARCHAR(10) NOT NULL,
    FOREIGN KEY (CustomerId) REFERENCES Customers(Id)
);
```

###**Вставляем тестовые данные:**
```sql
INSERT INTO Customers (Id, Name, Email) VALUES
(1, 'John Doe', 'john@example.com'),
(2, 'Jane Smith', 'jane@example.com');

INSERT INTO PageEvents (Id, PageName, CustomerId, Timestamp, Type) VALUES
(1, 'HomePage', 1, '2024-12-26 09:00:00', 'entry'),
(2, 'HomePage', 1, '2024-12-26 09:15:00', 'exit'),
(3, 'HomePage', 2, '2024-12-26 10:00:00', 'entry'),
(4, 'HomePage', 2, '2024-12-26 10:30:00', 'exit'),
(5, 'ContactPage', 1, '2024-12-26 11:00:00', 'entry'),
(6, 'ContactPage', 1, '2024-12-26 11:20:00', 'exit');
```


###Расчет среднего времени сессии**
Мы используем Common Table Expressions (CTE) для разделения данных на два набора:
- **`PageEntry`**: События входа на страницу.
- **`PageExit`**: События выхода с страницы.

Для каждой записи входа и выхода вычисляется уникальный ранг с помощью `ROW_NUMBER()`, чтобы правильно сопоставить события.

Затем мы рассчитываем разницу во времени между `EntryTime` и `ExitTime`, используя `DATEDIFF`, и вычисляем среднее время сессии.

**SQL-код:**
```sql
WITH PageEntry AS (
    SELECT
        PageName,
        CustomerId,
        Timestamp AS EntryTime,
        ROW_NUMBER() OVER (PARTITION BY PageName, CustomerId ORDER BY Timestamp) AS EntryRank
    FROM PageEvents
    WHERE Type = 'entry'
),
PageExit AS (
    SELECT
        PageName,
        CustomerId,
        Timestamp AS ExitTime,
        ROW_NUMBER() OVER (PARTITION BY PageName, CustomerId ORDER BY Timestamp) AS ExitRank
    FROM PageEvents
    WHERE Type = 'exit'
)
SELECT
    pe.PageName,
    AVG(CAST(DATEDIFF(MINUTE, pe.EntryTime, px.ExitTime) AS FLOAT)) AS AvgSessionTime
FROM PageEntry pe
JOIN PageExit px
    ON pe.PageName = px.PageName
    AND pe.CustomerId = px.CustomerId
    AND pe.EntryRank = px.ExitRank
GROUP BY pe.PageName;
```



#Задачи для самостоятельного решения


### Задача 1: Определение пользователей с повторным входом на страницы

#### Условие  
Даны таблицы `Customers` и `PageEvents` (описание таблиц — как в предыдущей задаче).  

1. Определить клиентов, которые заходили на одну и ту же страницу более одного раза за сутки.  
2. Для каждого такого клиента и страницы указать:  
   - Название страницы (`PageName`)  
   - Дату (`Date`)  
   - Количество входов на эту страницу за день.  

#### Ожидаемый результат  
| **CustomerId** | **PageName**   | **Date**       | **EntryCount** |
|-----------------|----------------|----------------|-----------------|
| 1              | HomePage       | 2024-12-26     | 2               |
| 2              | ContactPage    | 2024-12-27     | 3               |

---

### Задача 2: Поиск самой популярной последовательности страниц  

#### Условие  
Даны таблицы `Customers` и `PageEvents`.  

1. Определить самую популярную последовательность из двух подряд посещенных страниц (например, `HomePage → ContactPage`).  
2. Указать количество раз, когда эта последовательность встречалась.  
3. Если у клиента есть несколько подряд событий с одинаковыми страницами, такие последовательности учитывать только один раз.

#### Ожидаемый результат  
| **PageSequence**          | **Count** |
|----------------------------|-----------|
| HomePage → ContactPage     | 15        |
| ContactPage → AboutPage    | 10        |

---

### Задача 3: Среднее время между действиями  

#### Условие  
Даны таблицы `Customers` и `PageEvents`.  

1. Для каждого клиента вычислить среднее время между любыми двумя последовательными действиями на страницах (в секундах).  
2. Если у клиента было только одно действие, учитывать его не нужно.  

#### Ожидаемый результат  
| **CustomerId** | **AvgTimeBetweenActions (sec)** |
|-----------------|---------------------------------|
| 1              | 120.5                           |
| 2              | 95.0                            |

---

### Задача 4: Сессии с отсутствием выхода  

#### Условие  
Даны таблицы `Customers` и `PageEvents`.  

1. Найти всех клиентов, у которых есть события входа (`entry`), но нет соответствующего выхода (`exit`) для одной и той же страницы.  
2. Указать:  
   - Идентификатор клиента (`CustomerId`)  
   - Название страницы (`PageName`)  
   - Время последнего события входа на эту страницу.  

#### Ожидаемый результат  
| **CustomerId** | **PageName**   | **LastEntryTimestamp**  |
|-----------------|----------------|--------------------------|
| 1              | HomePage       | 2024-12-26 12:30:00     |
| 3              | ContactPage    | 2024-12-27 09:45:00     |

---

### Задача 5: Анализ активности по часам  

#### Условие  
Даны таблицы `Customers` и `PageEvents`.  

1. Разбить активность клиентов на временные интервалы по часу.  
2. Для каждого часа вычислить:  
   - Количество уникальных клиентов, которые посещали страницы.  
   - Общее количество действий (входов и выходов) на страницах.  

#### Ожидаемый результат  
| **Hour**       | **UniqueCustomers** | **TotalActions** |
|-----------------|---------------------|------------------|
| 09:00–10:00    | 10                  | 25               |
| 10:00–11:00    | 8                   | 15               |

---

### Задача 6: Самая активная страница  

#### Условие  
Даны таблицы `Customers` и `PageEvents`.  

1. Определить самую активную страницу за последние 7 дней.  
2. Активность страницы измеряется как общее количество событий (`entry` + `exit`).  

#### Ожидаемый результат  
| **PageName**   | **TotalEvents** |
|----------------|-----------------|
| HomePage       | 500            |

---

### Задача 7: Выявление сбоев в системе  

#### Условие  
Даны таблицы `Customers` и `PageEvents`.  

1. Считать, что произошел сбой, если:  
   - У одного клиента есть последовательность из двух событий `entry`, без промежуточного `exit`.  
   - У одного клиента есть последовательность из двух событий `exit`, без промежуточного `entry`.  
2. Вывести информацию о клиентах и страницах, где это произошло.  

#### Ожидаемый результат  
| **CustomerId** | **PageName**   | **ErrorType**    | **Timestamp**          |
|-----------------|----------------|------------------|------------------------|
| 1              | HomePage       | MissingExit      | 2024-12-26 10:00:00    |
| 2              | ContactPage    | MissingEntry     | 2024-12-26 11:30:00    |

---
